<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [102]:
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

load_dotenv()

host = os.getenv("84.201.134.129")
port = os.getenv("5432")
dbname = os.getenv("project_sql")
user = os.getenv("skillfactory")
password = os.getenv("cCkxxLVrDE8EbvjueeMedPKt")

db_url = f"postgresql+psycopg2://{'skillfactory'}:{'cCkxxLVrDE8EbvjueeMedPKt'}@{'84.201.134.129'}:{5432}/{'project_sql'}"
engine = create_engine(db_url)

In [103]:
query = f'''SELECT *
            FROM vacancies
'''
df = pd.read_sql(query, engine)
sql_df = df.copy()

print(sql_df.info())
# print(sql_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49197 entries, 0 to 49196
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           49197 non-null  int64  
 1   name         49197 non-null  object 
 2   key_skills   40741 non-null  object 
 3   schedule     49197 non-null  object 
 4   experience   49197 non-null  object 
 5   employment   49197 non-null  object 
 6   salary_from  21485 non-null  float64
 7   salary_to    12690 non-null  float64
 8   area_id      49197 non-null  int64  
 9   employer_id  49197 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 3.8+ MB
None


# Юнит 3. Предварительный анализ данных

**1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).**

In [54]:
query_3_1 = f'''SELECT
COUNT(v.id) AS count_vacancies
FROM vacancies AS v
'''

In [55]:
sql_df = pd.read_sql(query_3_1, engine)
print(sql_df)

   count_vacancies
0            49197


**2. Напишите запрос, который посчитает количество работодателей (таблица employers).**

In [56]:
query_3_2 = f'''SELECT
COUNT(e.id) AS count_employers
FROM employers AS e
'''

In [57]:
sql_df = pd.read_sql(query_3_2, engine)
print(sql_df)

   count_employers
0            23501


**3. Посчитате с помощью запроса количество регионов (таблица areas).**

In [58]:
query_3_3 = f'''SELECT
COUNT(a.id) AS count_areas
FROM areas AS a
'''

In [59]:
sql_df = pd.read_sql(query_3_3, engine)
print(sql_df)

   count_areas
0         1362


**4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).**

In [60]:
query_3_4 = f'''SELECT 
COUNT(i.id) AS count_industries
FROM industries AS i
'''

In [61]:
sql_df = pd.read_sql(query_3_4, engine)
print(sql_df)

   count_industries
0               294


***

**Вывод:** *Проведя предварительный анализ данных, мы узнали количество вакансий, работодателей, количество регионов и сфер деятельности.*

# Юнит 4. Детальный анализ вакансий

**1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.**

In [62]:
query_4_1 = f'''SELECT
a.id AS area_code,
a.name AS area_name,
COUNT(v.id) AS cnt

FROM vacancies AS v 
JOIN areas AS a ON v.area_id = a.id

GROUP BY a.id, a.name
ORDER BY cnt DESC

'''

In [63]:
sql_df = pd.read_sql(query_4_1, engine)
print(sql_df.head(5))

   area_code        area_name   cnt
0          1           Москва  5333
1          2  Санкт-Петербург  2851
2       1002            Минск  2112
3          4      Новосибирск  2006
4        160           Алматы  1892


**2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.**

In [64]:
query_4_2 = f'''SELECT
COUNT(*) AS cnt

FROM vacancies AS v 

WHERE v.salary_from IS NOT NULL
OR v.salary_to IS NOT NULL

'''

In [65]:
sql_df = pd.read_sql(query_4_2, engine)
print(sql_df)

     cnt
0  24073


**3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.**

In [66]:
query_4_3 = f'''SELECT
ROUND(AVG(salary_from)) AS avg_salary_from,
ROUND(AVG(salary_to)) AS avg_salary_to

FROM vacancies AS v 

WHERE v.salary_from IS NOT NULL
OR v.salary_to IS NOT NULL

'''

In [67]:
sql_df = pd.read_sql(query_4_3, engine)
print(sql_df)

   avg_salary_from  avg_salary_to
0          71065.0       110537.0


**4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.**


In [68]:
query_4_4 = f'''SELECT
v.schedule,
v.employment,
COUNT(v.id) AS cnt

FROM vacancies AS v 

GROUP BY v.schedule, v.employment
ORDER BY cnt DESC

'''

In [69]:
sql_df = pd.read_sql(query_4_4, engine)
print(sql_df.head(2))

           schedule        employment    cnt
0       Полный день  Полная занятость  35367
1  Удаленная работа  Полная занятость   7802


**5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.** 

In [70]:
query_4_5 = f'''SELECT
v.experience,
COUNT(*) AS cnt

FROM vacancies AS v 

GROUP BY v.experience
ORDER BY cnt ASC

'''

In [71]:
sql_df = pd.read_sql(query_4_5, engine)
print(sql_df)

           experience    cnt
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152


***

**Вывод:** *В детальном анализе вакансий мы узнали сколько вакансий существует в каждом регионе, в каких вакансиях заполнены поля с зарплатой. Также, мы нашли средние значения для нижней и верхней границ зарплатной вилки.*

# Юнит 5. Анализ работодателей

**1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.**

In [72]:
query_5_1 = f'''SELECT
e.name,
COUNT(v.id) AS vacancy_cnt

FROM employers AS e 
JOIN vacancies AS v ON e.id = v.employer_id

GROUP BY e.name
ORDER BY vacancy_cnt DESC
LIMIT 5
'''

In [73]:
sql_df = pd.read_sql(query_5_1, engine)
print(sql_df)

            name  vacancy_cnt
0         Яндекс         1933
1     Ростелеком          491
2       Тинькофф          444
3           СБЕР          428
4  Газпром нефть          331


**2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём. Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.**


In [74]:
query_5_2 = f'''SELECT 
a.name,
COUNT(DISTINCT v.id) AS vacation,
COUNT(DISTINCT e.id) AS companies
    
FROM areas AS a
    LEFT JOIN vacancies AS v ON a.id = v.area_id
    LEFT JOIN employers AS e ON a.id = e.area


WHERE v.area_id IS NULL

GROUP BY a.name
ORDER BY 3 DESC

LIMIT 5

'''

In [75]:
sql_df = pd.read_sql(query_5_2, engine)
print(sql_df)

                 name  vacation  companies
0              Россия         0        410
1           Казахстан         0        207
2  Московская область         0         75
3  Краснодарский край         0         19
4            Беларусь         0         18


**3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.**


In [76]:
query_5_3 = f'''SELECT
e.id AS employer_id,
e.name AS employer_name,
COUNT(DISTINCT v.area_id) AS areas_cnt
    
FROM employers AS e
    LEFT JOIN vacancies AS v ON e.id = v.employer_id

GROUP BY e.id, e.name
ORDER BY 3 DESC

'''

In [77]:
sql_df = pd.read_sql(query_5_3, engine)
print(sql_df)

       employer_id           employer_name  areas_cnt
0             1740                  Яндекс        181
1             2748              Ростелеком        152
2          5724811              Спецремонт        116
3          5130287  Поляков Денис Иванович         88
4          3682876               ООО ЕФИН          71
...            ...                     ...        ...
23496      5287515              Но Смокинг          0
23497      2839354        МАСТЕРСКАЯ CREED          0
23498      5774578           Dzing Finance          0
23499      5775069                    КОРС          0
23500           15             Наука-Связь          0

[23501 rows x 3 columns]


**4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.**

In [78]:
query_5_4 = f'''SELECT
COUNT(e.id) AS no_industry_cnt
    
FROM employers AS e
    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id

WHERE ei.industry_id IS NULL

'''

In [79]:
sql_df = pd.read_sql(query_5_4, engine)
print(sql_df)

   no_industry_cnt
0             8419


**5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.**

In [80]:
query_5_5 = f'''SELECT
e.name AS employer_name

FROM employers AS e
    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id

GROUP BY e.name
HAVING COUNT(ei.industry_id) = 4

ORDER BY e.name ASC

'''

In [81]:
sql_df = pd.read_sql(query_5_5, engine)
print(sql_df)

        employer_name
0        101 Интернет
1            21vek.by
2                2ГИС
3                  2К
4         4 пикселя +
...               ...
1131            ЮРИОН
1132  ЮТИП Технологии
1133           ЯКласс
1134            ЯрНео
1135    Яшин&Партнёры

[1136 rows x 1 columns]


**6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.**


In [82]:
query_5_6 = f'''SELECT
COUNT(e.id) AS cnt

FROM employers AS e
    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
    LEFT JOIN industries AS i ON ei.industry_id = i.id

WHERE i.name = 'Разработка программного обеспечения'

'''

In [83]:
sql_df = pd.read_sql(query_5_6, engine)
print(sql_df)

    cnt
0  3553


**7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку _Total_ с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.**

Список городов-милионников берём [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

In [84]:
url = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"
tables = pd.read_html(url, encoding = "utf-8")
cities_df = tables[0]
million_cities = cities_df.iloc[:, 1].tolist()
print(million_cities) 
cities_df = tables[0]
million_cities = cities_df.iloc[:, 1].tolist()
print(million_cities)

['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']
['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [85]:
query_5_7 = f'''WITH yandex_vacancies AS (
SELECT
    a.name AS city,
    COUNT(v.id) AS vacancy_cnt

FROM vacancies AS v
    LEFT JOIN areas AS a ON v.area_id = a.id
    LEFT JOIN employers AS e ON v.employer_id = e.id

WHERE e.name = 'Яндекс'
    AND a.name IN ({', '.join("'" + city + "'" for city in million_cities)})
    GROUP BY a.name
)

SELECT
    city,
    vacancy_cnt

FROM yandex_vacancies    

UNION ALL

SELECT 
    'Total' AS city, 
    SUM(vacancy_cnt) AS vacancy_cnt

FROM yandex_vacancies
ORDER BY vacancy_cnt DESC

'''

In [86]:
sql_df = pd.read_sql(query_5_7, engine)
print(sql_df)

               city  vacancy_cnt
0             Total        485.0
1            Москва         54.0
2   Санкт-Петербург         42.0
3      Екатеринбург         39.0
4   Нижний Новгород         36.0
5       Новосибирск         35.0
6           Воронеж         32.0
7         Краснодар         30.0
8            Самара         26.0
9               Уфа         26.0
10   Ростов-на-Дону         25.0
11            Пермь         25.0
12           Казань         25.0
13        Волгоград         24.0
14       Красноярск         23.0
15        Челябинск         22.0
16             Омск         21.0


***

**Вывод:** *Анализируя работодателей мы узнали топ-5 популярных работодателей по количеству вакансий, количество работодателей и вакансий для каждого региона, а также, количество регионов, в которых работодатель публикует свои вакансии. Узнали, что у 8419 (из 23501 возможных) работодателей не указана свера деятельности. С помощью парсинга мы сделали запрос на список городов-миллионников и вывели количество вакансий Яндекса для каждого города-миллионника.*

# Юнит 6. Предметный анализ

**1. Сколько вакансий имеет отношение к данным?**

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [87]:
query_6_1 = f'''
SELECT
    COUNT(DISTINCT v.id) AS vacancy_cnt

FROM vacancies AS v
    LEFT JOIN employers AS e ON v.employer_id = e.id
    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id
    LEFT JOIN industries AS i ON ei.industry_id = i.id

WHERE v.name ILIKE '%%данн%%' OR v.name ILIKE '%%data%%'

'''

In [88]:
sql_df = pd.read_sql(query_6_1, engine)
print(sql_df)

   vacancy_cnt
0         1771


**2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?**
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

*В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [89]:
query_6_2 = f'''
SELECT
    COUNT(DISTINCT v.id) AS junior_vacancy_cnt

FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name ILIKE '%%ML%%' AND v.name NOT ILIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%')
AND (
    v.name ILIKE '%%junior%%'
    OR v.experience ILIKE '%%Нет опыта%%'
    OR v.employment ILIKE '%%Стажировка%%'
)

'''

In [90]:
sql_df = pd.read_sql(query_6_2, engine)
print(sql_df)

   junior_vacancy_cnt
0                  51


**3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?**

*Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [91]:
query_6_3 = f'''
SELECT
    COUNT(DISTINCT v.id) AS ds_vacancy_cnt

FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name ILIKE '%%ML%%' AND v.name NOT ILIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%')
AND (
    v.key_skills ILIKE '%%SQL%%' OR v.key_skills ILIKE '%%postgres%%'
)

'''

In [92]:
sql_df = pd.read_sql(query_6_3, engine)
print(sql_df)

   ds_vacancy_cnt
0             229


**4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.**

*Это можно сделать помощью запроса, аналогичного предыдущему.*

In [93]:
query_6_4_1 = f'''
SELECT
    COUNT(DISTINCT v.id) AS python_vacancy_cnt

FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name ILIKE '%%ML%%' AND v.name NOT ILIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%')

'''
sql_df = pd.read_sql(query_6_4_1, engine)
print(sql_df)

   python_vacancy_cnt
0                 536


In [94]:
query_6_4 = f'''
SELECT
    COUNT(DISTINCT v.id) AS ds_vacancy_cnt

FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name ILIKE '%%ML%%' AND v.name NOT ILIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%')
AND (
    v.key_skills ILIKE '%%Python%%'
)

'''

In [95]:
sql_df = pd.read_sql(query_6_4, engine)
print(sql_df)

   ds_vacancy_cnt
0             357


**5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?**

Ответ округлите до двух знаков после точки-разделителя.

In [96]:
# Смотрим какие разделители используются

query_6_5_0 = f'''
SELECT 
    key_skills
FROM vacancies
'''
sql_df = pd.read_sql(query_6_5_0, engine)
print(sql_df.head(11))

                                           key_skills
0   Пользователь ПК\tРабота в команде\tРемонт ноут...
1   Средства криптографической защиты информации\t...
2     Spring Framework\tSQL\tHibernate ORM\tJava\tGit
3     Spring Framework\tSQL\tHibernate ORM\tJava\tGit
4                                                None
5   Корректура текстов\tГрамотность\tРабота в кома...
6        JavaScript\tTypeScript\tCSS3\tNode.js\tHTML5
7   Java SE\tSpring Framework\tSpring Boot\tGit\tS...
8   Linux\tАдминистрирование сетевого оборудования...
9   1С программирование\t1С: Предприятие 8\t1С: До...
10  PostgreSQL\tSQL\tLinux\t1С программирование\tM...


In [97]:
# CH(9) для \t табуляции

query_6_5 = f'''
SELECT
    ROUND(AVG(length(key_skills) - length(replace(key_skills, CHR(9),'')) + 1), 2) AS avg_skills_cnt

FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%'
    )
AND v.key_skills IS NOT NULL

'''

In [98]:
sql_df = pd.read_sql(query_6_5, engine)
print(sql_df)

   avg_skills_cnt
0            6.41


**6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).**

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [99]:
query_6_6 = f'''
SELECT 
    DISTINCT experience, 
    ROUND(AVG(
        CASE 
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL 
            THEN (salary_from + salary_to) / 2 
            ELSE COALESCE(salary_from, salary_to, 0) 
        END
    )) AS avg_salary
    
FROM vacancies AS v

WHERE (
    v.name ILIKE '%%data scientist%%' 
    OR v.name ILIKE '%%data science%%'
    OR v.name ILIKE '%%исследователь данных%%'
    OR v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%'
    OR v.name ILIKE '%%machine learning%%'
    OR v.name ILIKE '%%машинн%%обучен%%'
    )
AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)

GROUP BY experience

'''

In [100]:
sql_df = pd.read_sql(query_6_6, engine)
print(sql_df)

           experience  avg_salary
0           Нет опыта     74643.0
1  От 1 года до 3 лет    139675.0
2       От 3 до 6 лет    243115.0


**Вывод:** *В шестом юните мы провели предметный анализ, который позволил нам узнать сколько вакансий имеет отношения к данным. С помощью фильтрации мы нашли количество вакансий, которые относятся к DS и вакансии для DS в которых ключевыми навыками указаны SQL или postgres. Далее, с помощью ключевых навыков мы узнали, что ЯП Python является довольно популярным языком в вакансиях (где 536 вакансий DS и 357 вакансий, где учитывается ЯП Python).*

*Далее, мы определили среднее количество ключевых навыков для DS предварительно ознакомившись со способом разделения в таблицах. Среднее количество ключевых навыков составило 6,41.*

*Чтобы вычислить какую среднюю зарплату указывают для каждого типа опыта в DS мы рассматривали вакансии, где указаны хотя бы одно поле с зарплатой. С помощью функции CASE, которая сделала код гораздо компактнее и "читабельнее", мы поместили все три требующиеся от нас задачи в неё по порядку. Выяснили, что дата-сайентист с опытом работы от трёх до шести лет может рассчитывать на среднюю зарплату в районе 243115 д.ед.*

***

# Общий вывод по проекту

Подводя итоги проекта можно сделать вывод, что с помощью предварительного анализа мы можем найти информацию, которая нужна: посчитать количество вакансий или регионов, где определённая вакансия популярна, а также много других поверхностных деталей для ознакомления. 

Детально анализируя таблицы мы находим значения на уровне строк. Например количество вакансий, где заполнены хотя бы одно поле с зарплатой или найти средние значения для нижней/верхней границ зарплатной вилки. Также, мы провели детальный анализ работодателей. Узнали какие работодатели наиболее популярны, а также в каких регионах они публикуют свои вакансии. Если у работодателей не указана сфера их деятельности – здесь мы также прибегли к запросу о количестве таких работодателей. Также, полезно узнать какие именно работодатели не имеют сферы деятельности и посмотреть на их названия (*query_0_1*)

Также, с помощью парсинга, мы вытащили первую таблицу с сайта свободной энциклопедии *cities_df = tables[0]* и превратили её в список нужных нам городов. Узнали, что вакансии компании "Яндекс" представлены во всех городах-миллионниках, но больше всех их, конечно, в главных городах России - это Москва и Санкт-Петербург.

Предметный анализ показал нам как можно качественно и легко отфильтровать нужные критерии и находить нужные нам запросы.



In [101]:
query_0_1 = f'''SELECT
e.name AS employer_name
    
FROM employers AS e
    LEFT JOIN employers_industries AS ei ON e.id = ei.employer_id

WHERE ei.industry_id IS NULL

ORDER BY e.name ASC

'''

sql_df = pd.read_sql(query_0_1, engine)
print(sql_df)

                                          employer_name
0     1000 мелочей (ИП Гафиатуллина Гульшат Митарисо...
1                                             1001 Soft
2                                             100 печей
3                                              101 Плюс
4                                           1221Системс
...                                                 ...
8414           Ярославский завод упаковочных материалов
8415                           Ярославский филиал ПГУПС
8416    Ярославский ЦГМС - филиал ФГБУ Центральное УГМС
8417                                       Ярпож Казань
8418                            Яшнова Алена Викторовна

[8419 rows x 1 columns]
